此数据分析的数据源为近年来美国的交通事故数据集，其中的事故数据收集于2016年2月至2019年12月，覆盖了美国49个州。使用两种api提供流媒体交通事故数据，包括由各种实体捕获的交通数据，如美国和州交通部门、执法机构、交通摄像机和道路网络中的交通传感器等。目前，此数据集中大约有300万起事故记录。

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**数据源——DataSource https://www.kaggle.com/sobhanmoosavi/us-accidents**

**导入python包——Importing modules**

In [ ]:
import numpy as np #用法查询：https://numpy.org/doc/stable/user/quickstart.html#splitting-one-array-into-several-smaller-ones
import pandas as pd #用法查询：https://pandas.pydata.org/docs/user_guide/index.html
import matplotlib.pyplot as plt 
import matplotlib as mpl #用法查询：https://matplotlib.org/tutorials/index.html#introductory
import seaborn as sns #基于matplotlib的图形可视化python library，用法查询：http://seaborn.pydata.org/tutorial.html
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator #生成文字云,用法查询：http://amueller.github.io/word_cloud/auto_examples/index.html
from PIL import Image
import datetime

mpl.rcParams['font.sans-serif'] = ['SimHei']  #支持中文字体
plt.style.use('ggplot') #选择plot的呈现风格

**导入数据——Importing the dataset**

In [ ]:
df=pd.read_csv('/kaggle/input/us-accidents/US_Accidents_Dec19.csv') #df means DataFrame，数据框架

**显示数据——Displaying the dataset**

In [ ]:
df.head() #df.head()会将excel表格中的第一行看作列名，并默认输出之后的5行，在head后面的括号里面直接写你想要输出的行数也行，比如2，10，100之类的

**预览数据——Summary of Data**

In [ ]:
print('Rows     :',df.shape[0]) #df.shape[0]查看行数
print('Columns  :',df.shape[1]) #df.shape[1]查看列数
print('\nFeatures :\n     :',df.columns.tolist()) #将列名以list形式输出
print('\nMissing values    :',df.isnull().values.sum()) #判断缺失值数目
print('\nUnique values :  \n',df.nunique()) #返回各列唯一值的个数

In [ ]:
df.info() #显示基本信息

本数据集包含2974334条事故记录，共49列属性。其中有11817022条缺失值。

**找到类别型数据——Finding out columns with Catogerical Values**

In [ ]:
df.select_dtypes(exclude=['int','float']).columns #根据数据类型选择列，本行排除整数型、浮点型

In [ ]:
df['Description'].head() #输出“Description”列的前5行

**预览类别型数据——Displaying Catogerical values**

In [ ]:
print(df['Source'].unique()) #unique()以数组形式（numpy.ndarray）返回各列数据的所有唯一值
print(df['Description'].unique())
print(df['Timezone'].unique())
print(df['Amenity'].unique())

**相关性——Correlation**

In [ ]:
df.columns

In [ ]:
fig=sns.heatmap(df[['TMC','Severity','Start_Lat','End_Lat','Distance(mi)','Temperature(F)','Wind_Chill(F)',
                    'Humidity(%)','Pressure(in)','Visibility(mi)','Wind_Speed(mph)']].corr(),annot=True,
                cmap='RdYlGn',linewidths=0.25,annot_kws={'size':16}) 
#.corr()函数表示相关性，.heatmap()函数用来绘制热力图，linewidths：热力图矩阵之间的间隔大小，cmap：colormap，定义热力图颜色
#annot：annotate（评注），默认False，为True时，在每个方格写入数据，annot_kws：当annot为True时，可设置各个参数，包括大小，颜色，加粗，斜体字等

fig=plt.gcf() #得到当前Figure的引用
fig.set_size_inches(18,18) #设置图形大小
plt.xticks(fontsize=14) #x轴刻度字体大小
plt.yticks(fontsize=14)
plt.show()

**数据源——Source of Data**

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,8))
#定义plt.subplots()函数时，等式左边的f代表绘图窗口(Figure)；ax代表这个绘图窗口上的坐标系(axis)
#第一个1参数是子图的行数，第二个2参数是子图的列数（即1行2列，2个子图）；figsize用来设置图形的大小，a为图形的宽， b为图形的高，单位为英寸

df['Source'].value_counts().plot.pie(explode=[0.1,0.1,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
#value_counts函数用于统计dataframe或series中不同数或字符串出现的次数
#plot.pie函数绘制饼状图,explode:扇形之间的分离程度，默认为none，表示不分裂,autopct:扇形的数值标签,shadow：是否显示阴影，默认为false
#ax=ax[0]：让饼状图输出于第1个位置上
ax[0].set_title('Share of Sources') #设置标题
ax[0].set_ylabel('Count') #设置y轴标签

sns.countplot('Source',data=df,ax=ax[1],order=df['Source'].value_counts().index)
#countplot：显示条形图
#ax=ax[0]：让饼状图输出于第2个位置上，order：以值出现次数排序
ax[1].set_title('Count of Source')
plt.show()

由上图可知，在该数据集中，有74.1%的数据来源于MapQuest，24.5%是数据来源于Bing,剩余1.4%的数据来源于MapQuest-Bing。可知，大部分数据都是取自Mapquest——提供网上地图的美国网站，也有接近四分之一的数据取自Bing——微软的搜索引擎。

**严重程度——Severity:**
以1~4之间的分数显示事故的严重程度，1表示对交通的影响最小(即:事故只造成短暂延误)，4表示对交通造成重大影响(即：事故造成长期延误)。

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))

df['Severity'].value_counts().plot.pie(explode=[0.1,0.1,0.1,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Percentage Severity Distribution')
ax[0].set_ylabel('Count')

sns.countplot('Severity',data=df,ax=ax[1],order=df['Severity'].value_counts().index)
ax[1].set_title('Count of Severity')

plt.show()

由上图可知，67.0%，即三分之二所发生事故的严重程度都为2级，对交通造成了一定的延误；其次为3级，占比为29.8%，对交通造成了较长时间的延误；而1级和4级较少，分别占0.0%和3.1%。说明绝大部分事故都对交通造成了一定或较长时间的延误，较少出现造成短暂或长期延误的交通事故。

In [ ]:
plt.figure(figsize = (20,10))

df.Severity.value_counts(normalize=True).sort_index().plot.bar()
#normalize=True：以百分比的形式显示
#sort_index：默认按照行名排序
#plot.bar()绘制柱状图

plt.grid(False)
#绘制刻度线的网格线，False代表没有

plt.title('Severity')
plt.xlabel('Severity')
plt.ylabel('Fraction');

**东/西侧——Side:**

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))

df['Side'].value_counts().plot.pie(explode=[0.1,0.1,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Side types')
ax[0].set_ylabel('Count')

sns.countplot('Side',data=df,ax=ax[1],order=df['Side'].value_counts().index)
ax[1].set_title('Count of Side')

plt.show()

In [ ]:
df['Side'].unique()

**时区——Time Zone:**

In [ ]:
import matplotlib.pyplot as plt

from PIL import Image #导入图像处理库

%matplotlib inline
#将matplotlib的图表直接嵌入到Notebook之中

import numpy as np

img=np.array(Image.open('/kaggle/input/usa-timezone-map/usa-timezone-map.jpg'))
#Image.open函数：直接读取该路径指向的图片
#np.array(Image.open())打开图像并转化为数字矩阵，可以对图片进行复杂处理

fig=plt.figure(figsize=(30,10))
#plt.figure(figsize=(6,8))，表示figure 的大小为宽、长（单位为inch）

plt.imshow(img,interpolation='bilinear')
#imshow()函数实现热图绘制
#interplotation（插值）：用于改变图片的模糊度

plt.axis('off')
#关闭坐标轴，需置于 plt.show() 之前，plt.imshow() 之后

plt.show()

美国本土48州共有四个时区,从西往东依次为太平洋时区,山地时区,中部时区和东部时区。

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))

df['Timezone'].value_counts().plot.pie(explode=[0.1,0.1,0.1,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Accidents in Different Timezone')
ax[0].set_ylabel('Timezone')

sns.countplot('Timezone',data=df,ax=ax[1],order=df['Timezone'].value_counts().index)
ax[1].set_title('Accident Count Based on Timezone')

plt.show()

由上图可知，43.0%的交通事故都发生于美国的东部时区，27.1%的事故发生于太平洋时区，24.5%的事故发生于中部时区，5.4%的事故发生于山地时区。这是因为美国东部的人口密集、经济活动频繁，交通也较为发达，发生事故量自然也较多；其次为美国的西部-太平洋地区以及中部地区；而山地地区人口较少，经济相对落后，发生事故量较少。

**处理事故所需时间——Time taken to clear the traffic**

In [ ]:
start = pd.to_datetime(df.Start_Time, format='%Y-%m-%d %H:%M:%S') 
#pd.to_datetime函数数据（%Y-%m-%d %H:%M:%S（如2016-02-08 07:19:27））转换为时间类型，即datetime
end = pd.to_datetime(df.End_Time, format='%Y-%m-%d %H:%M:%S')

Duration = (end-start)

#提取每个事故的处理时间，以分钟为单位
T_D = 'Time_Duration(min)'
df[T_D] = Duration.astype('timedelta64[m]') #将时间类型转换为整数型

df.head()

In [ ]:
top20 = Duration.astype('timedelta64[m]').value_counts().nlargest(20)
#astype函数：实现变量类型转换，timedelta64表示时间差，[m]表示以单位minute表示，nlargest函数返回指定数目的最大值，计算数量最多的20个时间差的数量之和

print('top 20 accident durations correspond to {:.1f}% of the data'.format(top20.sum()*100/len(Duration)))
#top20.sum()用来计算数量最多的前20个时间差对应的数据个数和

(top20/top20.sum()).plot.bar(figsize=(18,8))
#.bar函数生成柱状图

plt.grid(False)
plt.title('Accident Duration [Minutes]')
plt.xlabel('Duration [minutes]')
plt.ylabel('Fraction')

从上面的曲线我们可以看到，处理所需时间为29.0分钟事故占比最高，超过40%，其次为44.0分钟和360.0分钟，都在15%左右，其他时间都小于10%，相对较少。这说明大多数事故需要不到1个小时的时间来进行处理，但也存在15%左右的事故需要360分钟才能解决。

In [ ]:
#输出数据
#df.to_csv(r'C:\Users\user\Desktop\数据分析项目\US Accidents\US_Accidents_Dec19_clean.csv',index=0)
#index=0表示不保存行索引

**不同州发生的事故量——Accident in different states**

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(15,8))

clr = ("blue", "forestgreen", "gold", "red", "purple",'cadetblue','hotpink','orange','darksalmon','brown')
df.State.value_counts().sort_values(ascending=False)[:10].sort_values().plot(kind='barh',color=clr,ax=ax[0])
#sort_values()函数将数据集依照某个字段中的数据进行排序，ascending=False表示降序排列
#plot函数用于画图，kind='barh'表示画横向条形图('line’:折线图（默认），‘bar’: 条形图，‘barh’:横向条形图，‘hist’:柱状图，‘box’:箱线图，
#‘kde’:Kernel的密度估计图，主要对柱状图添加Kernel概率密度线，‘density’:与‘kde’相同，‘pie’:饼图，‘scatter’:散点图
ax[0].set_title("Top 10 Accident Prone States",size=20)
ax[0].set_xlabel('States',size=18)


count=df['State'].value_counts()
groups=list(count.index)[:10]
groups.append('Other')

counts=list(count[:10])
counts.append(count.agg(sum)-count[:10].agg(sum))

type_dict=pd.DataFrame({"group":groups,"counts":counts})
#DataFrame是Python中Pandas库中的一种数据结构，它类似excel，是一种二维表
clr1=('brown','darksalmon','orange','hotpink','cadetblue','purple','red','gold','forestgreen','blue','plum')
qx = type_dict.plot(kind='pie', y='counts', labels=groups,colors=clr1,autopct='%1.1f%%', pctdistance=0.8, radius=1.2,ax=ax[1])
#pctdistance：锲形块数据标注的径向距离

plt.legend(loc=0, bbox_to_anchor=(1.15,0.4)) 
#legend函数用于定义图例位置，bbox_to_anchor用于指定图例在轴的位置
plt.subplots_adjust(wspace =0.2, hspace =1)
#wspace为子图之间的空间保留的宽度，平均轴宽的一部分，hspace为子图之间的空间保留的高度，平均轴高度的一部分

由上图可以看到事故最多发生在加利福尼亚州(CA)，占事故发生量的22.3%，其次为得克萨斯州(TX)和佛罗里达州(FL)，发生事故量最多的前十个州发生的事故量占整个美国发生事故量的65.9%。

In [ ]:
df_top_Severity_State = df.groupby('State').agg({'Severity': 'mean'}).sort_values('Severity').reset_index()
#reset_index()函数用于重置索引

plt.figure(figsize=(20,6))
sns.barplot(y="Severity", x="State", data=df_top_Severity_State.head(10))
#sns.barplot用于绘制条形图，x,y分别表述行、列名
plt.ioff()

由上图，可以看到亚利桑那州，南卡罗来纳和德克萨斯发生事故的严重程度最高。但是事故严重程度最高的前十个州发生事故的严重程度并没有显著差别。

**天气——Weather**

In [ ]:
plt.figure(figsize=(20,8))
df.groupby('Weather_Condition') \
        .size() \
        .sort_values(ascending = False) \
        .iloc[:5] \
        .plot.pie(explode=[0.1,0.1,0.1,0.1,0.1],autopct='%1.1f%%',shadow=True)
#iloc[]：取几行,例如,iloc[:5],取前5行
plt.ioff()

In [ ]:
fig, ax=plt.subplots(figsize=(20,8))

df['Weather_Condition'].value_counts().sort_values(ascending=False).head(5).plot.bar(width=0.5,edgecolor='k',align='center',linewidth=2)
#plot.bar函数中的width表示柱宽度(X轴方向)，edgecolor表示Bar边界线颜色，linewidth表示边界线宽
#align可选['left'(default) | 'center']，决定整个bar图分布，默认left表示默认从左边界开始绘制,center会将图绘制在中间位置

plt.xlabel('Weather_Condition',fontsize=20)
plt.ylabel('Number of Accidents',fontsize=20)
ax.tick_params(labelsize=20)
#tick_params函数的labelsize用于设置刻度线标签的字体大小
plt.title('5 Top Weather Condition for Accidents',fontsize=30)
plt.grid(False)
plt.ioff()

由上图可知，发生于晴朗天气下的事故数最多，占比为36.2%，这可能与晴朗天数较多以及天气晴朗时人们开车专注度不高有关。其余四种天气条件下发生事故量较为接近，都在10%与20%之间。

**事故发生地统计——The Analysis of Accidents Location**

In [ ]:
bool_cols = [col for col in df.columns if df[col].dtype == np.dtype('bool')]
booldf = df[bool_cols]
not_one_hot = booldf[booldf.sum(axis = 1) > 1]
print('There are {} non one hot metadata rows, which are {:.1f}% of the data'.format(len(not_one_hot),100*len(not_one_hot)/len(df)))

In [ ]:
bools = booldf.sum(axis=0)
#sum函数中axis=0表示按列相加

bools

In [ ]:
bools.plot.pie(autopct='%1.1f%%',shadow=True,figsize=(10,10))
plt.title('Proximity to Traffic Object')

由上图可以看到，接近一半的事故都发生在交通信号灯附近，其次就是交叉路口和十字路口，这三处地点是事故高发段，其他地段发生事故的比率都低于5%。

**事故统计——Accident Count**

In [ ]:
df['time'] = pd.to_datetime(df.Start_Time, format='%Y-%m-%d %H:%M:%S')
df = df.set_index('time')
#set_index用于添加索引
df.head()

In [ ]:
freq_text = {'D':'Daily','W':'Weekly','Y':'Yearly'}

plt.subplots(1,3,figsize=(21,7))
for i, (fr,text) in enumerate(freq_text.items(),1):
    plt.subplot(1,3,i)
    sample = df.ID['2016':].resample(fr).count()
    sample.plot(style='.')
    plt.title('Accidents, {} count'.format(text))
    plt.xlabel('Date')
    plt.ylabel('Accident Count');
#enumerate函数用于遍历序列中的元素以及它们的下标

由上图可以发现，事故发生数量呈逐年上升趋势，并且存在季节性变化趋势。

In [ ]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format="%Y/%m/%d %H:%M:%S")

#提取各事故发生年、月、日、小时和工作日
df['Year'] = df['Start_Time'].dt.year #year函数从日期型数据中获得当前年份
df['Month'] = df['Start_Time'].dt.strftime('%b') #strftime('%b')函数从日期型数据中获得当前月份
df['DayOfWeekNum'] = df['Start_Time'].dt.dayofweek #dayofweek将周几转换为数字，如monday转为0，sunday转为6
df['DayOfWeek'] = df['Start_Time'].dt.day_name() #day_name()函数从日期型数据中获得该日的星期名称
#df['Weekday']=df['Start_Time'].dt.strftime('%a')也可获得星期名称（前3位字母）
df['MonthDayNum'] = df['Start_Time'].dt.day #day函数从日期型数据中获得该日所在当前月份的排序
df['HourOfDay'] = df['Start_Time'].dt.hour #Hour函数从日期型数据中获得当前小时

df.head()

In [ ]:
fig, ax=plt.subplots(figsize=(20,8))
df['DayOfWeek'].value_counts(ascending=False).plot.bar(width=0.5,edgecolor='k',align='center',linewidth=2)
plt.xlabel('Day of the Week',fontsize=20)
plt.ylabel('Number of accidents',fontsize=20)
ax.tick_params(labelsize=20)
plt.title('Accident on Different Days of Week',fontsize=25)
plt.grid(False)
plt.ioff()

由上图可以发现，事故多发在工作日，较少发生在周末，这可能和人们的工作通勤以及出行习惯有关。

**美国地图背景文字云——WordCloud in the Context of USA Map**

In [ ]:
from wordcloud import WordCloud
#导入文字云包

plt.style.use('ggplot')
#设置绘图风格

Words = df["Description"].str.split("(").str[0].value_counts().keys()
#str.split()函数为分隔符，不支持正则及多个切割符号，不感知空格的数量
#keys函数用来返回字典中的所有的键

background_image = np.array(Image.open('/kaggle/input/usa-states-map/US-States-Map.png')) 
# Image.open() 方法读取背景图片,np.array 方法将图片转换为矩阵，便于进行复杂操作
img_colors = ImageColorGenerator(background_image) # 提取背景图片颜色

Stopwords = set(STOPWORDS) # 设置词云自带的英文过滤词,用来分割筛除文本中不需要的词汇，比如：a、an、the 这些

WC = WordCloud(scale=5,max_words=1000,mask = background_image,max_font_size = 36,stopwords = Stopwords,background_color="black").generate(" ".join(Words))
#scale：按照比例进行放大画布，如设置为1.5，则长和宽都是原来画布的1.5倍,max_words：设置最大现实字数
#colormap:matplotlib色图，可更改名称进而更改整体风格

WC.recolor(color_func=img_colors) # recolor() 则是根据图片色彩绘制词云文字颜色

plt.figure(figsize=(30,15))
plt.imshow(WC,interpolation="bilinear")
plt.axis("off")
plt.title("WordCloud in the Context of USA Map",color='R',Fontsize = 40)
plt.show()

In [ ]:
sns.factorplot('State','Severity',data=df)
#factorplot：对两个维度的数据绘制一组图
fig=plt.gcf()
#gcf（）函数：获取当前图形
fig.set_size_inches(20,7)
#set_size_inches用来设置图形的尺寸，单位为英寸
plt.show()

由上图可以看到，SD(南达科他州)，WY(怀俄明州)和AR(阿肯色州)发生的交通事故较为严重。

**散点图——Scatter Plot**

In [ ]:
plt.figure(figsize=(20,8))
Point = df[df.Severity<5]
Sca = Point.plot(kind='scatter', x='Start_Lng',y='Start_Lat',label='Severity',c='Severity',
                 cmap=plt.get_cmap('Oranges'),colorbar=True,alpha=0.3,figsize=(20,10))
#x,y：点位置，cmap指colormaps（色彩图），alpha透明度：介于0(透明)和1(不透明)之间。
Sca.legend()
plt.ioff()

由上图可以发现，美国东北部及密西西比河流域发生事故最为频繁，其次就是东南部、中部个别州以及西部的加州等地，这也和美国的人口分布、经济活动相吻合。

**选择用于机器学习算法预测的特征——Select a list of features for machine learning algorithms**

In [ ]:
#选择以下特征
feature_list=['Source','TMC','Severity','Start_Lng','Start_Lat','Distance(mi)','Side','City','County','State',
             'Timezone','Temperature(F)','Humidity(%)','Pressure(in)', 'Visibility(mi)', 'Wind_Direction','Weather_Condition',
             'Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming',
             'Traffic_Signal','Turning_Loop','Sunrise_Sunset','Hour','Weekday', 'Time_Duration(min)']